In [1]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMRegressor
import math
%matplotlib inline

pd.set_option('display.max_colwidth',1000)
pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

/Users/ljj/anaconda/envs/python3env/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


height has been deprecated.



-----------------

## 数据预处理

### 读取数据

In [2]:
train_data = pd.read_csv('./public.train.csv')
test_data = pd.read_csv('./public.test.csv')

df_result = pd.DataFrame()
df_result['ID'] = list(test_data['ID'])
special_missing_ID = test_data[test_data[(test_data == 0) | (test_data == 0.)].count(axis=1) > 13]['ID']

### 异常值处理

In [3]:
all_data = pd.concat([train_data, test_data], axis=0).sort_values(by='ID').reset_index().drop(['index'], axis=1)
bad_feature = ['ID', '功率A', '功率B', '功率C', '平均功率', '现场温度', '电压A', '电压B', '电压C', '电流B', '电流C', '转换效率', '转换效率A', '转换效率B', '转换效率C']
bad_index = all_data[bad_feature][
    (all_data[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std()) | 
    (all_data[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
].dropna(how='all').index

nn_bad_data = all_data.loc[np.concatenate([bad_index - 1, bad_index, bad_index + 1])].sort_values(by='ID', ascending=True).drop_duplicates()
bad_data = all_data.loc[bad_index].sort_values(by='ID', ascending=True)
len(bad_data)

53

In [4]:
# 上下记录均值替代异常值
for idx, line in bad_data.iterrows():
    ID = line['ID']
    col_index = line[bad_feature][ 
        (line[bad_feature] > all_data[bad_feature].mean() + 2 * all_data[bad_feature].std())| 
        (line[bad_feature] < all_data[bad_feature].mean() - 2 * all_data[bad_feature].std())
    ].index
    index = all_data[all_data['ID'] == ID].index
    
    before_offset = 1
    while (idx + before_offset)in bad_index:
        before_offset += 1

    after_offset = 1
    while (idx + after_offset) in bad_index:
        after_offset += 1
    
    replace_value = (all_data.loc[index - before_offset, col_index].values + all_data.loc[index + after_offset, col_index].values) / 2
    all_data.loc[index, col_index] = replace_value[0]

### 拆分数据

In [6]:
train_data = all_data.drop(all_data[all_data['ID'].isin(df_result['ID'])].index).reset_index().drop(['index'], axis=1)
test_data = all_data[all_data['ID'].isin(df_result['ID'])].drop(['发电量'], axis=1).reset_index().drop(['index'], axis=1)
len(train_data), len(test_data)

(9000, 8409)

### 去除重复值

In [7]:
dup_train_data = train_data.drop_duplicates(train_data.columns.drop('ID'), keep='first')

### 生成数据

In [9]:
def generate_train_data(train_data, test_data, poly=False, select=False):
    y = train_data['发电量']
    X = train_data.drop(['发电量','ID'], axis=1)
    sub_data = test_data.drop(['ID'], axis=1)

    if poly:
        from sklearn.preprocessing import PolynomialFeatures
        poly = PolynomialFeatures(degree=2, interaction_only=True)
        X = poly.fit_transform(X)
        sub_data = poly.transform(sub_data)
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    if select:
        from sklearn.feature_selection import SelectFromModel
        sm = SelectFromModel(GradientBoostingRegressor(random_state=2))
        X_train = sm.fit_transform(X_train, y_train)
        X_test = sm.transform(X_test)
        sub_data = sm.transform(sub_data)
        
    return X_train, X_test, y_train, y_test, sub_data

def cal_score(mse):
    if isinstance(mse, float):
        return 1 / (1 + math.sqrt(mse))
    else:
        return np.divide(1, 1 + np.sqrt(mse))

In [10]:
X_train, X_test, y_train, y_test, sub_data = generate_train_data(dup_train_data, test_data, poly=True, select=True)
cX_train, cX_test, cy_train, cy_test, _ = generate_train_data(train_data, test_data, poly=True, select=True)

--------

## Stacking Model

### Tree Models

In [12]:
all_X_train = np.concatenate([X_train, X_test])
all_y_train = np.concatenate([y_train, y_test])
c_all_X_train = np.concatenate([cX_train, cX_test])
c_all_y_train = np.concatenate([cy_train, cy_test])

In [11]:
xgbt1 = xgb.XGBRegressor(n_estimators=300, max_depth=3, random_state=2, n_jobs=8)
gbdt1 = GradientBoostingRegressor(n_estimators=300, max_depth=3, max_features='log2', random_state=2)
forest1 = RandomForestRegressor(n_estimators=100, max_features='log2', random_state=2, n_jobs=8)
lgb1 = LGBMRegressor(n_estimators=300, max_depth=3, random_state=2)

xgbt2 = xgb.XGBRegressor(n_estimators=200, random_state=3, n_jobs=8)
gbdt2 = GradientBoostingRegressor(n_estimators=200, max_features='sqrt', random_state=3)
forest2 = RandomForestRegressor(n_estimators=70, max_features='sqrt', random_state=3)
lgb2 = LGBMRegressor(n_estimators=200, random_state=3, n_jobs=8)

xgbt3 = xgb.XGBRegressor(n_estimators=300, random_state=4, n_jobs=8)
gbdt3 = GradientBoostingRegressor(n_estimators=300, max_features='sqrt', random_state=4)
forest3 = RandomForestRegressor(n_estimators=150, max_features='sqrt', random_state=4)
lgb3 = LGBMRegressor(n_estimators=300, random_state=4, n_jobs=8)

In [14]:
xgbt1.fit(all_X_train, all_y_train)
gbdt1.fit(all_X_train, all_y_train)
forest1.fit(all_X_train, all_y_train)
lgb1.fit(all_X_train, all_y_train)

xgbt2.fit(all_X_train, all_y_train)
gbdt2.fit(all_X_train, all_y_train)
forest2.fit(all_X_train, all_y_train)
lgb2.fit(all_X_train, all_y_train)

xgbt3.fit(c_all_X_train, c_all_y_train)
gbdt3.fit(c_all_X_train, c_all_y_train)
forest3.fit(c_all_X_train, c_all_y_train)
lgb3.fit(c_all_X_train, c_all_y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=300, n_jobs=8, num_leaves=31, objective=None,
       random_state=4, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### KNN models

In [20]:
from sklearn.neighbors import KNeighborsRegressor

knn1 = KNeighborsRegressor(n_neighbors=5)
knn1.fit(all_X_train, all_y_train)
scores_knn = cross_val_score(knn, all_X_train, all_y_train, cv=5, scoring='neg_mean_squared_error')
print(-scores_knn)
knn_avg = np.average(-scores_knn)
print('Average XGB - MSE:', knn_avg, ' - Score:', cal_score(-scores_knn), '\n')

knn2 = KNeighborsRegressor(n_neighbors=4)
knn2.fit(all_X_train, all_y_train)
scores_knn2 = cross_val_score(knn2, all_X_train, all_y_train, cv=5, scoring='neg_mean_squared_error')
print(-scores_knn2)
knn_avg2 = np.average(-scores_knn2)
print('Average XGB - MSE:', knn_avg2, ' - Score:', cal_score(-scores_knn2), '\n')

knn3 = KNeighborsRegressor(n_neighbors=4)
knn3.fit(c_all_X_train, c_all_y_train)
scores_knn3 = cross_val_score(knn3, c_all_X_train, c_all_y_train, cv=5, scoring='neg_mean_squared_error')
print(-scores_knn3)
knn_avg3 = np.average(-scores_knn3)
print('Average XGB - MSE:', knn_avg3, ' - Score:', cal_score(-scores_knn3))

[0.01937677 0.03540199 0.02753378 0.09990208 0.02136231]
Average XGB - MSE: 0.04071538665201484  - Score: [0.87780864 0.84164162 0.85768218 0.75983632 0.87247966] 

[0.02106734 0.0349659  0.0279258  0.09976482 0.02227228]
Average XGB - MSE: 0.04119922926004043  - Score: [0.87325116 0.84246586 0.85681716 0.75996175 0.87014101] 

[0.02715904 0.03592732 0.05523373 0.08671842 0.03033304]
Average XGB - MSE: 0.04707430925481936  - Score: [0.85851649 0.84065755 0.80970442 0.77251102 0.8516699 ]


### Stacking

In [22]:
regrs = [
    xgbt1, xgbt2, xgbt3,
    gbdt1, gbdt2, gbdt3,
    forest1, forest2, forest3,
    knn1, knn2, knn3
]

In [23]:
class Stacker(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
    
    # X: 原始训练集, y: 原始训练集真实值, predict_data: 原始待预测数据
    def fit_predict(self, X, y, predict_data):
        X = np.array(X)
        y = np.array(y)
        T = np.array(predict_data)

        folds = list(KFold(n_splits=self.n_splits, shuffle=False, random_state=2018).split(X, y))
        
        # 以基学习器预测结果为特征的 stacker的训练数据 与 stacker预测数据
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_predict = np.zeros((predict_data.shape[0], len(self.base_models)))
        
        for i, regr in enumerate(self.base_models):
            print('Base model:', str(regr).split('(')[0])
            S_predict_i = np.zeros((predict_data.shape[0], self.n_splits))
            
            for j, (train_idx, test_idx) in enumerate(folds):
                # 将X分为训练集与测试集
                X_train, y_train, X_test, y_test = X[train_idx], y[train_idx], X[test_idx], y[test_idx]
                print ('Fit fold', (j+1), '...')
                regr.fit(X_train, y_train)
                y_pred = regr.predict(X_test)                
                S_train[test_idx, i] = y_pred
                S_predict_i[:, j] = regr.predict(T)
            
            S_predict[:, i] = S_predict_i.mean(axis=1)

        nmse_score = cross_val_score(self.stacker, S_train, y, cv=5, scoring='neg_mean_squared_error')
        print('CV MSE:', -nmse_score)
        print('Stacker AVG MSE:', -nmse_score.mean(), 'Stacker AVG Score:', np.mean(np.divide(1, 1 + np.sqrt(-nmse_score))))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_predict)
        return res, S_train, S_predict

In [24]:
stacking_model = Ridge(alpha=0.387, copy_X=True, fit_intercept=False, solver='auto', random_state=2)
stacker = Stacker(5, stacking_model, regrs)

In [25]:
pred_stack, S_train_data, S_predict_data = stacker.fit_predict(all_X_train, all_y_train, sub_data)

Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: XGBRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: GradientBoostingRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: RandomForestRegressor
Fit fold 1 ...
Fit fold 2 ...
Fit fold 3 ...
Fit fold 4 ...
Fit fold 5 ...
Base model: KNeighborsRegressor
Fi

### Output

In [26]:
df_result['score'] = pred_stack

In [ ]:
index = df_result[df_result['ID'].isin(special_missing_ID)].index
df_result.loc[index, 'score'] = 0.379993053
df_result[df_result['ID'].isin(special_missing_ID)]

In [ ]:
df_result.to_csv('submit_stack_15_poly_select_dropdup_outlier.csv', index=False, header=False)